In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import h5py
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [9]:
file_electron = "/ElectronG.hdf5"
file_photon = "/PhotonG.hdf5"


with h5py.File(file_electron, "r") as f1:
    X_elec = np.array(f1['X'][:])
    y_elec = np.array(f1['y'][:])
with h5py.File(file_photon, "r") as f2:
    X_phot = np.array(f2['X'][:])
    y_phot = np.array(f2['y'][:])

In [10]:
print(X_elec.shape)
print(X_phot.shape)

(249000, 32, 32, 2)
(249000, 32, 32, 2)


In [11]:
X = np.append(X_elec, X_phot, axis=0)
y = np.append(y_elec, y_phot)
X.shape

(498000, 32, 32, 2)

In [12]:
X = np.swapaxes(X, 3,1) # Convenience of pytorch
X.shape

(498000, 2, 32, 32)

In [13]:
y = torch.as_tensor(y)

In [14]:
X = torch.from_numpy(X)

In [15]:
dataset = torch.utils.data.TensorDataset(X, y)

In [16]:
train_data,val_data = torch.utils.data.random_split(dataset,[int(len(dataset)*0.8),int(len(dataset)*0.2)])

In [17]:
trainset = torch.utils.data.DataLoader(train_data, batch_size=256, shuffle=True, num_workers=4, pin_memory=True)
valset = torch.utils.data.DataLoader(val_data, batch_size=256, shuffle=True, num_workers=4, pin_memory=True)

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from sklearn.metrics import roc_auc_score

# Residual Block with Skip Connections
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Skip connection
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = self.shortcut(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += identity  # Add skip connection
        out = self.relu(out)
        return out

# ResNet-15 Model
class ResNet15(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNet15, self).__init__()
        self.conv1 = nn.Conv2d(2, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)

        # Residual Block 1
        self.layer1 = self._make_layer(64, 64, num_blocks=2)
        # Residual Block 2
        self.layer2 = self._make_layer(64, 128, num_blocks=2, stride=2)
        # Residual Block 3
        self.layer3 = self._make_layer(128, 256, num_blocks=2, stride=2)
        # Residual Block 4
        self.layer4 = self._make_layer(256, 512, num_blocks=2, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, in_channels, out_channels, num_blocks, stride=1):
        layers = []
        layers.append(ResidualBlock(in_channels, out_channels, stride))
        for _ in range(1, num_blocks):
            layers.append(ResidualBlock(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Focal Loss Class
class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=0.25):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction="none")
        p_t = torch.exp(-ce_loss)  # Probabilities of the correct class
        focal_loss = self.alpha * (1 - p_t) ** self.gamma * ce_loss
        return focal_loss.mean()

# Initialize model, loss function, and optimizer
model = ResNet15(num_classes=2)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = FocalLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# Learning Rate Scheduler (Cosine Annealing)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

# Training Loop
for epoch in range(25):
    running_loss = 0.0
    val_loss = 0.0
    all_y_true = torch.zeros(0, dtype=torch.long, device=device)
    all_y_pred = torch.zeros(0, dtype=torch.long, device=device)
    all_y_true_val = torch.zeros(0, dtype=torch.long, device=device)
    all_y_pred_val = torch.zeros(0, dtype=torch.long, device=device)

    model.train()  # Set model to training mode
    for i, data in enumerate(trainset, 0):
        X, y = data
        if torch.cuda.is_available():
            X = X.cuda()
            y = y.cuda()

        optimizer.zero_grad()
        outputs = model(X)
        preds = outputs
        preds = preds.detach()
        _, preds = torch.max(preds, 1)

        # Concatenate true labels and predictions
        all_y_true = torch.cat((all_y_true, y.view(-1)))
        all_y_pred = torch.cat((all_y_pred, preds.view(-1)))

        loss = criterion(outputs, y.long())
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 500 == 499:
            auc_score = roc_auc_score(all_y_true.cpu().numpy(), all_y_pred.cpu().numpy())
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 500:.3f} auc_score_train: {auc_score}')
            running_loss = 0.0

    # Step the learning rate scheduler
    scheduler.step()

    # Validation Loop
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        for j, data in enumerate(valset, 0):
            X_val, y_val = data
            if torch.cuda.is_available():
                X_val = X_val.cuda()
                y_val = y_val.cuda()

            val_outputs = model(X_val)
            loss_val = criterion(val_outputs, y_val.long())

            _, val_preds = torch.max(val_outputs, 1)

            all_y_true_val = torch.cat((all_y_true_val, y_val.view(-1)))
            all_y_pred_val = torch.cat((all_y_pred_val, val_preds.view(-1)))

            val_loss += loss_val.item()

    auc_score_val = roc_auc_score(all_y_true_val.cpu().numpy(), all_y_pred_val.cpu().numpy())
    print(f'val_loss: {val_loss / j:.3f} auc_score_val: {auc_score_val}')


[1,   500] loss: 0.043 auc_score_train: 0.6103598057629804
[1,  1000] loss: 0.040 auc_score_train: 0.6230494841565468
[1,  1500] loss: 0.039 auc_score_train: 0.6356545488863681
val_loss: 0.040 auc_score_val: 0.6453792292147739
[2,   500] loss: 0.038 auc_score_train: 0.6843034482250739
[2,  1000] loss: 0.037 auc_score_train: 0.6897033718333743
[2,  1500] loss: 0.037 auc_score_train: 0.6940015462506525
val_loss: 0.039 auc_score_val: 0.6753297768086768
[3,   500] loss: 0.036 auc_score_train: 0.708283987522305
[3,  1000] loss: 0.036 auc_score_train: 0.710052619259536
[3,  1500] loss: 0.036 auc_score_train: 0.711585176382536
val_loss: 0.036 auc_score_val: 0.7139051881622035
[4,   500] loss: 0.036 auc_score_train: 0.7157151141479
[4,  1000] loss: 0.036 auc_score_train: 0.7158256576064843
[4,  1500] loss: 0.036 auc_score_train: 0.7163601800815206
val_loss: 0.036 auc_score_val: 0.7162618995799862
[5,   500] loss: 0.036 auc_score_train: 0.7192093498593334
[5,  1000] loss: 0.036 auc_score_train:

In [32]:
# Save the entire model (including architecture and weights)
torch.save(model, 'resnet15_model.pth')


In [34]:
from google.colab import files
files.download('resnet15_model.pth')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>